# Dependências

In [ ]:
# Install Google Cloud SDK components (typically run once per environment)
# Note: These are shell commands, not Python code, hence the '!' prefix in Jupyter notebooks
!pip install gcloud  # Installs the Google Cloud CLI tools
!gcloud auth application-default login  # Sets up application default credentials for GCP access

# Import required Python packages for data processing and Google Cloud operations
import pandas as pd         # Primary data analysis library (DataFrames, Series)
import numpy as np          # Numerical computing (arrays, math operations)
import time                 # Time measurement and delays
import os                   # Operating system interactions (files, paths)
import pandas_gbq           # Pandas-BigQuery integration (direct DataFrame transfers)
from google.cloud import bigquery  # Official BigQuery client library (detailed operations)
import glob                 # File path pattern matching (finding files by patterns)
import openpyxl             # Excel file reading/writing (.xlsx format support)
import csv                  # CSV file reading/writing
import re                   # Regular expressions (text pattern matching)

# Tratamento

In [ ]:
# Load data from Excel file (Social Assistance sheet)
df = pd.read_excel("Base_Estadic_2023(2).xlsx", sheet_name='Assistência Social')

# Select relevant columns and preview data
df = df[['Sigla UF', 'Cod UF', 'Nome UF', 'EASS01', 'EASS03', 'EASS04', 'EASS05', 'EASS06']]
df['ano'] = 2023  # Add year column

# Rename columns for clarity
df = df.rename(columns={
    'Sigla UF': 'sigla_uf',
    'Cod UF': 'cod_uf',
    'Nome UF': 'uf',
    'EASS01': 'caracterizacao_orgao_gestor',
    'EASS03': 'genero',
    'EASS04': 'idade',
    'EASS05': 'cor_raca',
    'EASS06': 'grau_instrucao'
})

# Clean and standardize data
df['caracterizacao_orgao_gestor'] = df['caracterizacao_orgao_gestor'].str.title()
df['cor_raca'] = np.where(df['cor_raca'] == 'Pardo', 'Parda', df['cor_raca'])

# Handle missing/refused data
for col in ['caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao']:
    df[col] = np.where(df[col].isin(['Recusa', 'Não Informou', 'Não informou']),
                     'Sem dados',
                     df[col])

# Convert age to numeric and handle missing values
df['idade'] = np.where(df['idade'].isin(['Recusa', 'Não informou']), np.nan, df['idade'])
df['idade'] = pd.to_numeric(df['idade'])

# Create age groups
limites = [18, 30, 50, 65, 100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']
df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias, right=False)

# Select final columns
df = df[['ano', 'sigla_uf', 'cod_uf', 'uf', 'caracterizacao_orgao_gestor',
         'genero', 'faixa_etaria', 'cor_raca', 'grau_instrucao']]

# Standardize education levels
dict_esco = {
    'Ensino superior completo': 'Até Ensino Superior Completo',
    'Especialização': 'Até Pós Graduação ou Mestrado',
    'Mestrado': 'Até Pós Graduação ou Mestrado',
    'Doutorado': 'Até Doutorado'
}
df = df.replace({'grau_instrucao': dict_esco})

# Preview final data
df.head()
df.info()

# Upload

In [ ]:
# Define the BigQuery table schema with field descriptions in Portuguese
schema = [
    bigquery.SchemaField('ano', 'INTEGER', description='Ano da apuração daquele dado'),
    bigquery.SchemaField('sigla_uf', 'STRING', description='Sigla da UF'),
    bigquery.SchemaField('cod_uf', 'INTEGER', description='Código do IBGE da UF'),
    bigquery.SchemaField('uf', 'STRING', description='Nome da UF'),
    bigquery.SchemaField('caracterizacao_orgao_gestor', 'STRING',
                       description='Caracterização do órgão no qual o gestor está'),
    bigquery.SchemaField('genero', 'STRING',
                       description='Gênero autodeclarado ou não'),
    bigquery.SchemaField('faixa_etaria', 'STRING',
                       description='faixa etária da observação'),
    bigquery.SchemaField('cor_raca', 'STRING',
                       description='Raça/cor da pessoa observada'),
    bigquery.SchemaField('grau_instrucao', 'STRING',
                       description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')
]

# Initialize BigQuery client connection
# Using the specified project 'repositoriodedadosgpsp'
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create reference to target dataset 'cargos_lideranca'
dataset_ref = client.dataset('cargos_lideranca')

# Create reference to target table with standardized naming convention:
# Format: SOURCE_descriptive_name_version (ESTADIC_perfil_gestor_assistencia_social_tipo_orgao_v4)
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_assistencia_social_tipo_orgao_v4')

# Configure the load job with our schema definition
job_config = bigquery.LoadJobConfig(schema=schema)

# Execute the load job to transfer DataFrame to BigQuery
# Parameters:
#   df - The pandas DataFrame containing our processed data
#   table_ref - The destination table reference
#   job_config - Our schema configuration
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the job to complete before proceeding
# This blocks execution until the upload finishes
job.result()